In [1]:
import pandas as pd
import numpy as np

In [2]:
contests = pd.read_csv('data/WorkingData.csv')

In [3]:
print(contests.columns)

Index(['Unnamed: 0', 'ContestId', 'SportName', 'VariantName', 'GameSet',
       'ContestName', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST',
       'ContestPayoutDatetimeEST', 'EntryFeeAmount', 'TotalPrizeAmount',
       'MaxNumberPlayers', 'MaxEntriesPerUser', 'Entries', 'DistinctUsers',
       'Contest_Group', 'NumGames', 'DraftablePlayersInSet',
       'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'is_vault'],
      dtype='object')


## We need:
Add threshold, Number of entries 4 hours out etc. 


In [4]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

In [5]:
# Get rid of cheats
features = contests.columns.drop('Entries').drop('DistinctUsers')
# Get rid of Useless 
#drop('ContestId')
features = features.drop('Unnamed: 0').drop('GameSet').drop('ContestName').drop('is_vault')
contests = contests[features].dropna()

In [6]:
contests.head()

,ContestId,SportName,VariantName,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,Contest_Group,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition
0,7963004,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,0.25,25000.0,117500,100,Headliner,15,910.0,668396,2000.0,23665
1,7963437,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,2.00,25000.0,14375,100,Booster,15,910.0,668396,20.0,1250
2,8296224,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,100.00,5000.0,55,1,SingleEntry,15,910.0,668396,1100.0,11
3,8296544,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,50.00,5000.0,111,11,FeaturedDoubleUp,15,910.0,668396,100.0,50
4,8494213,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,5.00,300000.0,69000,200,Headliner,15,910.0,668396,25000.0,13805


### Categorizing

In [7]:
def categorizeFeature(df, name):
    return df.assign(newCol=lb_make.fit_transform(categoricalDF[name])).rename(columns={'newCol': name+'_code'})

In [8]:
categoricalFeatures = ['SportName', 'VariantName', 'Contest_Group']
categoricalDF = contests[['ContestId', 'SportName', 'VariantName', 'Contest_Group']]
for f in categoricalFeatures:
    categoricalDF = categorizeFeature(categoricalDF, f)
categoricalDF = categoricalDF.drop(categoricalFeatures, axis=1)

categoricalDF.head()

,ContestId,SportName_code,VariantName_code,Contest_Group_code
0,7963004,10,1,7
1,7963437,10,1,2
2,8296224,10,1,13
3,8296544,10,1,5
4,8494213,10,1,7


### Scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [10]:
def scaleFeature(df, name):
    return df.assign(newCol=scaler.fit_transform(df[[name]])).rename(columns={'newCol': name+'_Scaled'})

In [11]:
scaleFeatures = ['EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 
                       'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition']
scaleDF = contests[['ContestId', 'EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 
                       'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition']]
for f in scaleFeatures:
    scaleDF = scaleFeature(scaleDF, f)
scaleDF = scaleDF.drop(scaleFeatures, axis=1)
scaleDF.head()

,ContestId,EntryFeeAmount_Scaled,TotalPrizeAmount_Scaled,MaxNumberPlayers_Scaled,MaxEntriesPerUser_Scaled,NumGames_Scaled,DraftablePlayersInSet_Scaled,PaidUsersInDraftGroup_Scaled,TopPrize_Scaled,MaxPayoutPosition_Scaled
0,7963004,0.000005,0.0025,0.149810,9.900000e-08,0.222222,0.582524,0.891705,0.000999,0.122835
1,7963437,0.000068,0.0025,0.018326,9.900000e-08,0.222222,0.582524,0.891705,0.000009,0.006483
2,8296224,0.003568,0.0005,0.000068,0.000000e+00,0.222222,0.582524,0.891705,0.000549,0.000052
3,8296544,0.001782,0.0005,0.000139,1.000000e-08,0.222222,0.582524,0.891705,0.000049,0.000254
4,8494213,0.000175,0.0300,0.087973,1.990000e-07,0.222222,0.582524,0.891705,0.012499,0.071654


### Time
time data : month, day of month, day of week, hours


In [12]:
timeFeatures = ['ContestId', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST', 'ContestPayoutDatetimeEST']
mapper = {'ContestStartDatetimeEST' : 'Start', 'ContestEndDatetimeEST' : 'End', 'ContestPayoutDatetimeEST': 'Payout'}
timeDF = contests[timeFeatures]
formatString = '%Y-%m-%d %H:%M:%S'

In [13]:
timeDF.head()

,ContestId,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST
0,7963004,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
1,7963437,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
2,8296224,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
3,8296544,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
4,8494213,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000


In [14]:
def mapTime(df, name, colHeader):
    miniDF = df[['ContestId', name]]
    d = {
        'ContestId': [],
        'Minute' : [],
        'Hour' : [],
        'Day' : [],
        'Month' : [],
        'Year' : [],
        'DayOfWeek' : [],
        'DayOfYear' : [],
        'WeekOfYear' : []
    }
    for index, row in miniDF.iterrows():
        time =  pd.to_datetime(row[name], format=formatString)
        d['ContestId'].append(row['ContestId'])
        d['Minute'].append(int(time.strftime('%M')))
        d['Hour'].append(int(time.strftime('%H')))
        d['Day'].append(int(time.strftime('%d')))
        d['Month'].append(int(time.strftime('%m')))
        d['Year'].append(int(time.strftime('%Y')))
        d['DayOfWeek'].append(int(time.strftime('%w')))
        d['DayOfYear'].append(int(time.strftime('%-j')))
        d['WeekOfYear'].append(int(time.strftime('%U')))
        
    renamer = {}
    for a in d:
        if a != 'ContestId':
            renamer[a] = colHeader+"_"+a       
    newFrame = pd.DataFrame(data=d).rename(columns=renamer)  
    miniDF = pd.merge(miniDF, newFrame, on="ContestId")
    return miniDF

This next one takes ages, hence the 'timeDFBackup=timesDF'

In [15]:
timesDF = timeDF[['ContestId']]
timeDF.head()
for f in mapper:
    timesDF = pd.merge(timesDF, mapTime(timeDF, f, mapper[f]), on="ContestId").drop(columns=[f], axis=1)
timeDFBackup=timesDF

### Scale Time Data

In [16]:
timesDF.tail()

,ContestId,Start_Minute,Start_Hour,Start_Day,Start_Month,Start_Year,Start_DayOfWeek,Start_DayOfYear,Start_WeekOfYear,End_Minute,...,End_DayOfYear,End_WeekOfYear,Payout_Minute,Payout_Hour,Payout_Day,Payout_Month,Payout_Year,Payout_DayOfWeek,Payout_DayOfYear,Payout_WeekOfYear
381174,58300381,5,22,31,7,2018,2,212,30,10,...,212,30,16,2,1,8,2018,3,213,30
381175,58300538,5,22,31,7,2018,2,212,30,10,...,212,30,16,2,1,8,2018,3,213,30
381176,58300644,5,22,31,7,2018,2,212,30,10,...,212,30,16,2,1,8,2018,3,213,30
381177,58300656,5,22,31,7,2018,2,212,30,10,...,212,30,16,2,1,8,2018,3,213,30
381178,58300706,10,22,31,7,2018,2,212,30,10,...,212,30,16,2,1,8,2018,3,213,30


In [17]:
scaleTimeFeatures = timesDF.columns.drop('ContestId')
scaleTimeFeatures

Index(['Start_Minute', 'Start_Hour', 'Start_Day', 'Start_Month', 'Start_Year',
       'Start_DayOfWeek', 'Start_DayOfYear', 'Start_WeekOfYear', 'End_Minute',
       'End_Hour', 'End_Day', 'End_Month', 'End_Year', 'End_DayOfWeek',
       'End_DayOfYear', 'End_WeekOfYear', 'Payout_Minute', 'Payout_Hour',
       'Payout_Day', 'Payout_Month', 'Payout_Year', 'Payout_DayOfWeek',
       'Payout_DayOfYear', 'Payout_WeekOfYear'],
      dtype='object')

In [18]:
for f in scaleTimeFeatures:
    timesDF = scaleFeature(timesDF, f)
timesDF = timesDF.drop(scaleTimeFeatures, axis=1)

In [19]:
timesDF.head()

,ContestId,Start_Minute_Scaled,Start_Hour_Scaled,Start_Day_Scaled,Start_Month_Scaled,Start_Year_Scaled,Start_DayOfWeek_Scaled,Start_DayOfYear_Scaled,Start_WeekOfYear_Scaled,End_Minute_Scaled,...,End_DayOfYear_Scaled,End_WeekOfYear_Scaled,Payout_Minute_Scaled,Payout_Hour_Scaled,Payout_Day_Scaled,Payout_Month_Scaled,Payout_Year_Scaled,Payout_DayOfWeek_Scaled,Payout_DayOfYear_Scaled,Payout_WeekOfYear_Scaled
0,7963004,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
1,7963437,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
2,8296224,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
3,8296544,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
4,8494213,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113


### Splitting Scaling into 2 columns

In [20]:
import math

In [21]:
def circularizeFeature(df, feature):
    fStr = feature[:-len('scaled')]
    df = df.assign(X= lambda c: np.sin(df[feature]*2*math.pi)).assign(Y= lambda c: np.cos(df[feature]*2*math.pi))
    df = df.rename(columns={'X': fStr+'X', 'Y': fStr+'Y'})
    return df

In [22]:
circularFeatures = timesDF.columns
circleDF = timesDF[circularFeatures]
for f in circularFeatures:
    if f != 'ContestId':
        circleDF = circularizeFeature(circleDF, f)
circleDF = circleDF.drop(circularFeatures.drop('ContestId'), axis=1)

In [23]:
circleDF.head()

,ContestId,Start_Minute_X,Start_Minute_Y,Start_Hour_X,Start_Hour_Y,Start_Day_X,Start_Day_Y,Start_Month_X,Start_Month_Y,Start_Year_X,...,Payout_Month_X,Payout_Month_Y,Payout_Year_X,Payout_Year_Y,Payout_DayOfWeek_X,Payout_DayOfWeek_Y,Payout_DayOfYear_X,Payout_DayOfYear_Y,Payout_WeekOfYear_X,Payout_WeekOfYear_Y
0,7963004,0.0,1.0,-0.398401,-0.917211,0.587785,-0.809017,-0.989821,-0.142315,0.0,...,-0.989821,-0.142315,0.0,1.0,0.866025,-0.5,-0.958718,-0.284359,-0.947326,-0.32027
1,7963437,0.0,1.0,-0.398401,-0.917211,0.587785,-0.809017,-0.989821,-0.142315,0.0,...,-0.989821,-0.142315,0.0,1.0,0.866025,-0.5,-0.958718,-0.284359,-0.947326,-0.32027
2,8296224,0.0,1.0,-0.398401,-0.917211,0.587785,-0.809017,-0.989821,-0.142315,0.0,...,-0.989821,-0.142315,0.0,1.0,0.866025,-0.5,-0.958718,-0.284359,-0.947326,-0.32027
3,8296544,0.0,1.0,-0.398401,-0.917211,0.587785,-0.809017,-0.989821,-0.142315,0.0,...,-0.989821,-0.142315,0.0,1.0,0.866025,-0.5,-0.958718,-0.284359,-0.947326,-0.32027
4,8494213,0.0,1.0,-0.398401,-0.917211,0.587785,-0.809017,-0.989821,-0.142315,0.0,...,-0.989821,-0.142315,0.0,1.0,0.866025,-0.5,-0.958718,-0.284359,-0.947326,-0.32027


### Calculating threshold

In [24]:
calcFeatures = ['ContestId', 'TotalPrizeAmount','EntryFeeAmount']
calcDF = contests[calcFeatures]
calcDF = calcDF.assign(SuccessThreshold= lambda x:(calcDF['TotalPrizeAmount'].astype(float)/(calcDF['EntryFeeAmount'].astype(float))).astype(int))
calcDF = calcDF[['ContestId', 'SuccessThreshold']]

In [25]:
calcDF = scaleFeature(calcDF, 'SuccessThreshold')
threshDF = calcDF[['ContestId','SuccessThreshold']]
scaledThreshDF = calcDF[['ContestId','SuccessThreshold_Scaled']]
# calcDF
# calcDF = calcDF.drop(columns=['SuccessThreshold'], axis=1)

In [26]:
calcDF.tail()

,ContestId,SuccessThreshold,SuccessThreshold_Scaled
372206,58300381,150,0.000224
372207,58300538,50,0.000074
372208,58300644,50,0.000074
372209,58300656,50,0.000074
372210,58300706,50,0.000074


### Calculating Time-Series Data

In [27]:
seriesDF = contests[['ContestId', ]]
cIds = seriesDF['ContestId']
cTimeDF = timeDFBackup[['ContestId','Start_Month', 'Start_Year', 'End_Month', 'End_Year']]
seekHours = [0, 1, 2, 3, 4, 6, 8]
# seekHours = [0, 1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 20, 24, 36, 48]

In [28]:
def seriesByMonthAndYear(year, month):
    seriesDF = pd.read_csv('data/WPI/'+ str(year) + '-' + str(month) + '.csv', header=None)
    seriesDF.columns = ['ContestId', 'SecondsRemaining', 'Entries']
    return seriesDF

In [29]:
#times should be an array of hours in ascending order i.e.[2, 4, 6, 8]
def getTimeSplits(df, times):
    entrySums = []
    for hour in times:
        entrySums.append(df[(df['SecondsRemaining'] >= hour*60*60)].Entries.sum())
    return entrySums

In [30]:
def getTimeSeriesData(ContestId):
    contestDF = cTimeDF.loc[cTimeDF['ContestId']==ContestId]
    
    sMonth = (contestDF.Start_Month).item()
    if(sMonth <= 9):
        sMonth = "0"+str(sMonth)
    sYear = (contestDF.Start_Year).item()
    eMonth = (contestDF.End_Month).item()
    if(eMonth <= 9):
        eMonth = "0"+str(eMonth)
    eYear = (contestDF.End_Year).item()
    
    seriesDF = seriesByMonthAndYear(sYear, sMonth)
    if sMonth != eMonth:
        seriesDF = seriesDF.append(seriesByMonthAndYear( eYear, eMonth))
    seriesDF = seriesDF.loc[seriesDF['ContestId']==ContestId]
    timeSplits = getTimeSplits(seriesDF, seekHours)
    hourData = {'ContestId':ContestId}
    for i in range(len(seekHours)):
        hours = seekHours[i]
        entries = timeSplits[i]
        hourData[str(hours)+"HoursOut"] = [entries]
    newSeriesDF = pd.DataFrame(data=hourData)
    return newSeriesDF

In [35]:
seriesDF = pd.DataFrame()
for cId in cIds:
    seriesDF = seriesDF.append(getTimeSeriesData(cId), ignore_index=True)

ValueError: can only convert an array of size 1 to a Python scalar

### Scale Series to Threshold

In [ ]:
scaledSeriesDF = pd.merge(threshDF, seriesDF, on="ContestId")

In [ ]:
scaledSeriesDF.head()

In [ ]:
def percentifyFeature(df, numerator, denomenator):
    return df.assign(newCol=lambda x: df[numerator].astype(int)/df[denomenator].astype(int)).rename(columns={'newCol': numerator+'_Percent'})

In [ ]:
percentifyFeatures = seriesDF.columns.drop('ContestId')

In [ ]:
for f in percentifyFeatures:
    scaledSeriesDF = percentifyFeature(scaledSeriesDF, f, 'SuccessThreshold')
scaledSeriesDF = scaledSeriesDF.drop(seriesDF.columns.drop('ContestId'), axis=1).drop(['SuccessThreshold'], axis=1)

### Merge altogether

# RANDOM THOUGHT- LETS DETERMINE WHEN MOST CONTESTS REACH THRESHOLD

In [ ]:
mergable = [categoricalDF, scaleDF, circleDF, scaledThreshDF, scaledSeriesDF]

In [ ]:
mergedDF = contests[['ContestId']]
for df in mergable:
    mergedDF = pd.merge(mergedDF, df, on='ContestId')

In [ ]:
for i in mergedDF.columns:
    print(i)

In [ ]:
mergedDF.tail()

In [ ]:
mergedDF.to_csv("data/PCA_Features.csv")